In [ ]:
!pip install ipython-sql

In [35]:

%load_ext sql
%sql sqlite:///combined_data.db


The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [36]:
import pandas as pd
import sqlite3

# Replace with the actual names of your uploaded CSV files
df1 = pd.read_csv('/content/application_details_granted_2018_updated.csv')
df2 = pd.read_csv('/content/application_details_granted_2019_updated.csv')
df3 = pd.read_csv('/content/application_details_granted_2020_updated.csv')
df4 = pd.read_csv('/content/application_details_granted_2021_updated.csv')
df5 = pd.read_csv('/content/application_details_granted_2022_updated.csv')

# Concatenate the DataFrames
df = pd.concat([df1, df2, df3, df4, df5], axis=0)

# Create sqLite connection and save combined to SQL
conn = sqlite3.connect('combined_data.db')
df.to_sql('combined_table', con=conn, if_exists='replace', index=False)

# a cursor object
cursor = conn.cursor()

# Query
cursor.execute("SELECT * FROM combined_table LIMIT 100;")
results = cursor.fetchall()

# Optionally, you can print the results
for row in results:
    print(row)


('01/2018', '05/01/2018', 'Delhi', 'PCT NATIONAL PHASE APPLICATION', '1002/DELNP/2010', '2010-02-15', '"PHASE ANALYSIS MEASUREMENT APPARATUS AND METHOD"', 'PHYSICS', 'Science', 2010, 56.0, 19.0, '19/04/2023')
('01/2018', '05/01/2018', 'Delhi', 'PCT NATIONAL PHASE APPLICATION', '10068/DELNP/2008', '2008-12-03', '"PATCH FOR MODIFICATION OF THE POWER OF AN OPTICAL COMPONENT"', 'ELECTRONICS', 'Technology', 2008, 20.0, 34.0, '19/04/2023')
('01/2018', '05/01/2018', 'Delhi', 'DIVISIONAL PCT NATIONAL PHASE APPLICATION', '10069/DELNP/2008', '2008-12-03', '"A METHOD FOR DRIVING OR PUSHING PILES"', 'CIVIL', 'Industrial', 2008, 37.0, 7.0, '19/04/2023')
('01/2018', '05/01/2018', 'Delhi', 'PCT NATIONAL PHASE APPLICATION', '10300/DELNP/2012', '2012-11-27', 'TARGET POINT RECOGNITION METHOD AND SURVEYING INSTRUMENT', 'PHYSICS', 'Science', 2012, 29.0, 15.0, '19/04/2023')
('01/2018', '05/01/2018', 'Delhi', 'PCT NATIONAL PHASE APPLICATION', '10397/DELNP/2012', '2012-11-29', '"ADSORBENT FOR FEED AND PRODUC

In [37]:
cursor.execute("SELECT * FROM combined_table ORDER BY rowid DESC LIMIT 100;")
results_desc = cursor.fetchall()
print("Last 100 rows in descending order by rowid:")
for row in results_desc:
    print(row)

Last 100 rows in descending order by rowid:
('52/2022', '30/12/2022', 'Delhi', 'PCT NATIONAL PHASE APPLICATION', '9779/DELNP/2012', '2012-11-12', 'CUTTING ELEMENTS FOR EARTH-BORING TOOLS INCLUDING SACRIFICIAL\r\nSTRUCTURES IN INDENTATIONS IN CUTTING FACES OF DIAMOND TABLES,\r\nAND RELATED EARTH-BORING TOOLS AND METHODS', 'MECHANICAL', 'Industrial', 2012, 30.0, 20.0, '19/04/2023')
('52/2022', '30/12/2022', 'Mumbai', 'ORDINARY APPLICATION', '977/MUM/2015', '2015-03-24', 'SYSTEM AND METHOD FOR THE GENERATION OF THREE DIMENSIONAL MEDICAL RAPID PROTOTYPING COMPUTER TOMOGRAPHY PROTOCOL (MRCP) MODEL UTILIZING A DIGITAL RADIOGRAPHIC IMAGE', 'MECHANICAL', 'Industrial', 2015, 26.0, 9.0, '19/04/2023')
('52/2022', '30/12/2022', 'Kolkata', 'PCT NATIONAL PHASE APPLICATION', '977/KOLNP/2014', '2014-05-07', 'SYSTEM AND METHOD FOR A DISTRIBUTED VIRTUAL DESKTOP INFRASTRUCTURE', 'COMPUTER', 'Technology', 2014, 26.0, 20.0, '19/04/2023')
('52/2022', '30/12/2022', 'Delhi', 'PCT NATIONAL PHASE APPLICATION', 

In [38]:
cursor.execute("SELECT COUNT(*) AS total_rows FROM combined_table;")
total_rows = cursor.fetchone()[0]
print(f"Total rows: {total_rows}")

Total rows: 120085


In [39]:
cursor.execute("PRAGMA TABLE_INFO('combined_table');")
total_columns = len(cursor.fetchall())
print(f"Total columns: {total_columns}")


Total columns: 13


In [40]:
# Step 1: Create invention_category table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS invention_category (
        FIELD_OF_INVENTION TEXT PRIMARY KEY NOT NULL,
        CATEGORY_OF_INVENTION TEXT
    )
''')

# Step 2: Create filing_date table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS filing_date (
        DATE_OF_FILING TEXT PRIMARY KEY NOT NULL,
        FILING_YEAR INTEGER
    )
''')

# Step 3: Create invention_information table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS invention_information (
        TITLE_OF_INVENTION TEXT PRIMARY KEY NOT NULL,
        DATE_OF_FILING TEXT NOT NULL,
        FIELD_OF_INVENTION TEXT NOT NULL,
        NO_OF_PAGES INTEGER,
        NO_OF_CLAIMS INTEGER,
        FOREIGN KEY (DATE_OF_FILING) REFERENCES filing_date(DATE_OF_FILING),
        FOREIGN KEY (FIELD_OF_INVENTION) REFERENCES invention_category(FIELD_OF_INVENTION)
    )
''')

# Step 4: Create application_information table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS application_information (
        APPLICATION_NUMBER TEXT PRIMARY KEY NOT NULL,
        APPLICATION_TYPE_DESC TEXT,
        IPO_LOCATION TEXT,
        PUBLICATION_DATE TEXT,
        TITLE_OF_INVENTION TEXT NOT NULL,
        FOREIGN KEY (TITLE_OF_INVENTION) REFERENCES invention_information(TITLE_OF_INVENTION)
    )
''')

# Commit the table creation commands
conn.commit()
print("Tables created successfully with specified relationships.")


Tables created successfully with specified relationships.


In [41]:
# Split data into different DataFrames based on table columns
# Prepare the invention_category data, dropping rows with null values and keeping unique FIELD_OF_INVENTION values
invention_category_df = df[['FIELD_OF_INVENTION', 'CATEGORY_OF_INVENTION']].drop_duplicates()
invention_category_df = invention_category_df.dropna(subset=['FIELD_OF_INVENTION', 'CATEGORY_OF_INVENTION'])
# Remove duplicates based on FIELD_OF_INVENTION, keeping only the first occurrence
invention_category_df = invention_category_df.drop_duplicates(subset=['FIELD_OF_INVENTION'])

# Insert cleaned data into the invention_category table
invention_category_df.to_sql('invention_category', conn, if_exists='replace', index=False)

# 2. Filing Date
filing_date_df = df[['DATE_OF_FILING', 'FILING_YEAR']].drop_duplicates()

# 3. Invention Information
invention_information_df = df[['TITLE_OF_INVENTION', 'DATE_OF_FILING', 'FIELD_OF_INVENTION', 'NO_OF_PAGES', 'NO_OF_CLAIMS']].drop_duplicates()
invention_information_df = invention_information_df.dropna(subset=['TITLE_OF_INVENTION', 'DATE_OF_FILING', 'FIELD_OF_INVENTION', 'NO_OF_PAGES', 'NO_OF_CLAIMS'])

# 4. Application Information
application_information_df = df[['APPLICATION_NUMBER', 'APPLICATION_TYPE_DESC', 'IPO_LOCATION', 'PUBLICATION_DATE', 'TITLE_OF_INVENTION']].drop_duplicates()

# Insert data into each table using .to_sql
invention_category_df.to_sql('invention_category', conn, if_exists='append', index=False)
filing_date_df.to_sql('filing_date', conn, if_exists='replace', index=False)
invention_information_df.to_sql('invention_information', conn, if_exists='replace', index=False)
application_information_df.to_sql('application_information', conn, if_exists='replace', index=False)

# Commit the transactions
conn.commit()
print("Data inserted successfully.")

Data inserted successfully.


In [58]:
print("Invention Category Table:")
%sql SELECT * FROM invention_category;

Invention Category Table:
 * sqlite:///combined_data.db
Done.


FIELD_OF_INVENTION,CATEGORY_OF_INVENTION
PHYSICS,Science
ELECTRONICS,Technology
CIVIL,Industrial
CHEMISTRY,Science
ELECTRICAL,Industrial
MECHANICAL,Industrial
FOOD,Others
BIOTECHNOLOGY,Science
POLYMER,Science
BIOCHEMISTRY,Science


In [59]:
print("\nFiling Date Table:")
%sql SELECT * FROM filing_date LIMIT 10;


Filing Date Table:
 * sqlite:///combined_data.db
Done.


DATE_OF_FILING,FILING_YEAR
2010-02-15,2010
2008-12-03,2008
2012-11-27,2012
2012-11-29,2012
2013-09-12,2013
2007-06-06,2007
2011-03-31,2011
2011-06-01,2011
2010-01-11,2010
2009-01-21,2009


In [60]:
print("\nInvention Information Table:")
%sql SELECT * FROM invention_information LIMIT 10;


Invention Information Table:
 * sqlite:///combined_data.db
Done.


TITLE_OF_INVENTION,DATE_OF_FILING,FIELD_OF_INVENTION,NO_OF_PAGES,NO_OF_CLAIMS
"""PHASE ANALYSIS MEASUREMENT APPARATUS AND METHOD""",2010-02-15,PHYSICS,56.0,19.0
"""PATCH FOR MODIFICATION OF THE POWER OF AN OPTICAL COMPONENT""",2008-12-03,ELECTRONICS,20.0,34.0
"""A METHOD FOR DRIVING OR PUSHING PILES""",2008-12-03,CIVIL,37.0,7.0
TARGET POINT RECOGNITION METHOD AND SURVEYING INSTRUMENT,2012-11-27,PHYSICS,29.0,15.0
"""ADSORBENT FOR FEED AND PRODUCTS PURIFICATION IN BENZENE SATURATION PROCESS""",2012-11-29,CHEMISTRY,17.0,10.0
'A METHOD OF HEATING CLEANING SOLVENTS TO ACIDIC RANGE FOR REMOVING MILL SCALE RUST FORMED DURING MANUFACTURING STORAGE AND FABRICATION AND BOILER OPERATIONAL STAGES OF SUB CRITICAL SUPER CRITICAL DRUM TYPE/ONCE THROUGH BOILERS',2013-09-12,CHEMISTRY,13.0,6.0
SINGLE USE LOCK SYSTEM,2007-06-06,CIVIL,10.0,5.0
“A NETWORK ELEMENT FOR A TELECOMMUNICATION SYSTEM”,2011-03-31,ELECTRICAL,13.0,8.0
“PROCESS FOR REMOVING BY-PRODUCTS FROM N-VINYLAMIDES”,2011-06-01,CHEMISTRY,12.0,17.0
CRANKCASE VENTILATION SYSTEM WITH PUMPED SCAVENGED OIL,2010-01-11,MECHANICAL,23.0,30.0


In [62]:
print("\nApplication Information Table:")
%sql SELECT * FROM application_information LIMIT 5;


Application Information Table:
 * sqlite:///combined_data.db
Done.


APPLICATION_NUMBER,APPLICATION_TYPE_DESC,IPO_LOCATION,PUBLICATION_DATE,TITLE_OF_INVENTION
1002/DELNP/2010,PCT NATIONAL PHASE APPLICATION,Delhi,05/01/2018,"""PHASE ANALYSIS MEASUREMENT APPARATUS AND METHOD"""
10068/DELNP/2008,PCT NATIONAL PHASE APPLICATION,Delhi,05/01/2018,"""PATCH FOR MODIFICATION OF THE POWER OF AN OPTICAL COMPONENT"""
10069/DELNP/2008,DIVISIONAL PCT NATIONAL PHASE APPLICATION,Delhi,05/01/2018,"""A METHOD FOR DRIVING OR PUSHING PILES"""
10300/DELNP/2012,PCT NATIONAL PHASE APPLICATION,Delhi,05/01/2018,TARGET POINT RECOGNITION METHOD AND SURVEYING INSTRUMENT
10397/DELNP/2012,PCT NATIONAL PHASE APPLICATION,Delhi,05/01/2018,"""ADSORBENT FOR FEED AND PRODUCTS PURIFICATION IN BENZENE SATURATION PROCESS"""


In [63]:
%sql SELECT ai.APPLICATION_NUMBER, ai.APPLICATION_TYPE_DESC, ai.IPO_LOCATION, ai.PUBLICATION_DATE, ii.TITLE_OF_INVENTION, ii.NO_OF_PAGES, ii.NO_OF_CLAIMS FROM application_information ai JOIN invention_information ii ON ai.TITLE_OF_INVENTION = ii.TITLE_OF_INVENTION LIMIT 5;


 * sqlite:///combined_data.db
Done.


APPLICATION_NUMBER,APPLICATION_TYPE_DESC,IPO_LOCATION,PUBLICATION_DATE,TITLE_OF_INVENTION,NO_OF_PAGES,NO_OF_CLAIMS
1002/DELNP/2010,PCT NATIONAL PHASE APPLICATION,Delhi,05/01/2018,"""PHASE ANALYSIS MEASUREMENT APPARATUS AND METHOD""",56.0,19.0
10068/DELNP/2008,PCT NATIONAL PHASE APPLICATION,Delhi,05/01/2018,"""PATCH FOR MODIFICATION OF THE POWER OF AN OPTICAL COMPONENT""",20.0,34.0
10069/DELNP/2008,DIVISIONAL PCT NATIONAL PHASE APPLICATION,Delhi,05/01/2018,"""A METHOD FOR DRIVING OR PUSHING PILES""",37.0,7.0
10300/DELNP/2012,PCT NATIONAL PHASE APPLICATION,Delhi,05/01/2018,TARGET POINT RECOGNITION METHOD AND SURVEYING INSTRUMENT,29.0,15.0
10397/DELNP/2012,PCT NATIONAL PHASE APPLICATION,Delhi,05/01/2018,"""ADSORBENT FOR FEED AND PRODUCTS PURIFICATION IN BENZENE SATURATION PROCESS""",17.0,10.0


In [ ]:
	SINGLE USE LOCK SYSTEM

In [54]:
%sql SELECT * FROM invention_information WHERE TITLE_OF_INVENTION = 'SINGLE USE LOCK SYSTEM';


 * sqlite:///combined_data.db
Done.


TITLE_OF_INVENTION,DATE_OF_FILING,FIELD_OF_INVENTION,NO_OF_PAGES,NO_OF_CLAIMS
SINGLE USE LOCK SYSTEM,2007-06-06,CIVIL,15.0,5.0


In [56]:
%sql UPDATE invention_information SET NO_OF_PAGES = 10 WHERE TITLE_OF_INVENTION = 'SINGLE USE LOCK SYSTEM';


 * sqlite:///combined_data.db
1 rows affected.


[]

In [57]:
%sql SELECT * FROM invention_information WHERE TITLE_OF_INVENTION = 'SINGLE USE LOCK SYSTEM';


 * sqlite:///combined_data.db
Done.


TITLE_OF_INVENTION,DATE_OF_FILING,FIELD_OF_INVENTION,NO_OF_PAGES,NO_OF_CLAIMS
SINGLE USE LOCK SYSTEM,2007-06-06,CIVIL,10.0,5.0
